# Samples discharging only

In [1]:
import sys

sys.path.append('..')

In [2]:
import os
import gc
import yaml
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from joblib import Parallel, delayed
from utils import optimize_dtypes, get_files, int16_repr

In [3]:
BASE_PATH = '../../'

CONFIG_DIR = os.path.join(BASE_PATH, 'config')
STORAGE_DIR = os.path.join(BASE_PATH, 'storage')

config = yaml.load(open(os.path.join(CONFIG_DIR, 'env.yml')),
                   Loader=yaml.FullLoader)

In [4]:
model = 'samples'
model_storage = os.path.join(STORAGE_DIR, model)
output_storage = os.path.join(STORAGE_DIR, 'output')

if not os.path.exists(output_storage):
    os.makedirs(output_storage)

In [5]:
files = get_files(os.path.join(model_storage, '*.parquet'))

In [6]:
def compute(filepath):
    df = pd.read_parquet(filepath)
    df = df[df['charger'] == 'unplugged']
    
    return df.drop(['battery_state', 'charger'], axis=1)

In [7]:
data = Parallel(n_jobs=-1)(delayed(compute)(f) for f in files[12:])

In [8]:
df = pd.concat(data, axis=0, ignore_index=True)
del data[:]
del data
gc.collect()

0

In [ ]:
df = df.sort_values(by=['device_id', 'timestamp'])

In [ ]:
df.to_parquet(os.path.join(output_storage, 'unplugged.parquet'), index=False, compression='brotli')